In [1]:
!pip install -q wandb

You should consider upgrading via the 'c:\users\erlend\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [1]:
from preprocessing import create_train_test, min_max_scale, df_to_xy, read_file, lag_features
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

# Hyperparameter tuning


import wandb
from wandb.keras import WandbCallback
import os
os.environ["WANDB_NOTEBOOK_NAME"] = "C:/Users/Erlend/Google Drive/NTNU/5. Klasse/Prosjektoppgave/Git folder/LSTM-for-option-pricing"
wandb.login(key = "3cae81eb56be3190be5bb48c571e69933071df69")

wandb: Currently logged in as: erlendrygg (avogadro). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Erlend/.netrc


True

In [2]:
#Variables
first_year = 2019
last_year = 2021
split_date ="2021-01-01"

epochs = 300

features = ["Moneyness", "Ttl", "R", "Volatility"]
seq_length = 5
num_features = len(features)



In [3]:
file = f"./data/processed_data/{first_year}-{last_year}.csv"

df_read = read_file(file)
print(df_read)
df_read.info()
print(df_read)

         Unnamed: 0  Quote_date Expire_date  Underlying_last  Strike      Ask  \
0           1354913  2019-01-02  2019-01-04          2509.98   800.0  1711.10   
1           1354914  2019-01-02  2019-01-04          2509.98   900.0  1611.40   
2           1354915  2019-01-02  2019-01-04          2509.98  1000.0  1511.40   
3           1354916  2019-01-02  2019-01-04          2509.98  1050.0  1462.19   
4           1354917  2019-01-02  2019-01-04          2509.98  1100.0  1412.20   
...             ...         ...         ...              ...     ...      ...   
5123793     6521988  2021-12-31  2024-12-20          4766.39  8400.0   300.00   
5123794     6521989  2021-12-31  2024-12-20          4766.39  8600.0   300.00   
5123795     6521990  2021-12-31  2024-12-20          4766.39  8800.0   300.90   
5123796     6521991  2021-12-31  2024-12-20          4766.39  9000.0   300.00   
5123797     6521992  2021-12-31  2024-12-20          4766.39  9200.0   300.00   

             Bid  Bid_strik

In [4]:
# Splitting dataset

df_read_lags = lag_features(df_read, features, seq_length)

df_train_orginal, df_test_orginal = create_train_test(df_read_lags, split_date)

train_x_org, train_y_org, = df_to_xy(df_train_orginal, num_features, seq_length)
test_x_org, test_y_org = df_to_xy(df_test_orginal, num_features, seq_length)

train_x_scaled, test_x_scaled = min_max_scale(train_x_org, test_x_org)
train_y_scaled, test_y_scaled = min_max_scale(train_y_org, test_y_org)

train_x_scaled = np.reshape(train_x_scaled, (len(train_x_scaled), seq_length, num_features))
test_x_scaled = np.reshape(test_x_scaled, (len(test_x_scaled), seq_length, num_features))

print(f"Train_x shape: {train_x_scaled.shape}, train_y shape: {train_y_scaled.shape}")
print(f"Test_x shape: {test_x_scaled.shape}, test_y shape: {test_y_scaled.shape}")

Train_x shape: (2785226, 5, 4), train_y shape: (2785226, 2)
Test_x shape: (1845482, 5, 4), test_y shape: (1845482, 2)


In [5]:
sweep_configuration = {
    'method': 'bayes',
    'name': 'Erlend-test-2',
    'metric': {
        'goal': 'minimize', 
        'name': 'val_loss'
		},
    'parameters': {
        "units": {'values': [20, 40, 60, 80]},
        "input_dropout": {"distribution": "uniform",
            'max': 0.4, 'min': 0},
        "recurrent_dropout": {
            "distribution": "uniform",
            'max': 0.4, 'min': 0},
        "output_dropout": {
            "distribution": "uniform",
            'max': 0.4, 'min': 0},
        "lr": {
            "distribution": "uniform",
            'max': 0.01, 'min': 0.00001},
        "clip_norm": {
            "distribution": "uniform",
            'max': 1, 'min': 0.00001},
        "layers": {'values': [2, 3, 4, 5]},
        "minibatch_size": {'values': [512, 1024, 2048, 4096]}
    }
}


In [7]:
# Initialize sweep and creating sweepID
#sweep_id = wandb.sweep(sweep=sweep_configuration, project="LSTM option pricing")
#sweep8: sweep_id = 7finyjz4
#sweep10 
sweep_id = "iysyrskx"

In [8]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Input, Dropout
from keras import backend as K
from tensorflow.keras.optimizers import Adam
import keras as KER
from sklearn.model_selection import train_test_split
from keras.activations import linear, relu

In [9]:
def create_model(config):
  """Builds an LSTM model of minimum 2 layers sequentially from a given config dictionary"""
  model = Sequential()


  seq_length = 5
  num_features =  4
  model.add(LSTM(
    units = config.units,
    activation = relu,
    input_shape = (seq_length, num_features),
    dropout = config.input_dropout,
    recurrent_dropout = config.recurrent_dropout,
    return_sequences = True
  )) 

  model.add(Dropout(
    rate=config.output_dropout)
  )

  for i in range(config.layers - 2):
    model.add(LSTM(
      units = config.units,
      activation = relu,
      dropout = config.input_dropout,
      recurrent_dropout = config.recurrent_dropout,
      return_sequences = True))
    
    model.add(Dropout(
      rate=config.output_dropout)
    )

  model.add(LSTM(
    units = config.units,
    activation = relu,
    dropout = config.input_dropout,
    recurrent_dropout = config.recurrent_dropout,
    return_sequences = False
  ))

  model.add(Dropout(
    rate=config.output_dropout)
  )

  model.add(Dense(
    units = 2,
    activation = relu
  ))  

  model.compile(
    optimizer = Adam(
      learning_rate = config.lr,
      clipnorm = config.clip_norm
    ),
    loss = "mse",
    metrics = ["accuracy"]
  )

  return model

In [10]:
from keras.callbacks import EarlyStopping


def trainer(train_x = train_x_scaled, train_y = train_y_scaled, config = None):
    # Initialize a new wandb run
    with wandb.init(config=sweep_configuration):

        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config

        model = create_model(config)

        early_stopping = EarlyStopping(
            monitor='val_loss',
            mode='min',
            min_delta=1e-6,
            patience=10,
        )
        
        wandb_callback = WandbCallback(
            monitor='val_loss',
            mode='min',
            save_model=False
        )

        shuffle = np.random.permutation(len(train_x))
        train_x, train_y = train_x[shuffle], train_y[shuffle]
        model.fit(
            train_x,
            train_y,
            batch_size = config.minibatch_size,
            validation_split = 0.3,
            epochs = epochs,
            callbacks = [early_stopping, wandb_callback] 
        )
        


In [11]:
wandb.agent(sweep_id=sweep_id, function=trainer, project="LSTM option pricing", count = 3)

wandb: Agent Starting Run: 2xe4uajq with config:
wandb: 	clip_norm: 0.819431820115507
wandb: 	dropout: 0.2274176002660815
wandb: 	layers: 3
wandb: 	lr: 0.004020372903708056
wandb: 	minibatch_size: 1024
wandb: 	recurrent_dropout: 0.33975860608965225
wandb: 	units: 60


Epoch 1/300
1904/1904 [==============================] - 234s 119ms/step - loss: 6.0237e-05 - accuracy: 0.5826 - val_loss: 6.3704e-05 - val_accuracy: 0.3874
Epoch 2/300
1904/1904 [==============================] - 252s 132ms/step - loss: 1.2342e-05 - accuracy: 0.5791 - val_loss: 6.7742e-05 - val_accuracy: 0.8760
Epoch 3/300
1904/1904 [==============================] - 494s 259ms/step - loss: 1.1919e-05 - accuracy: 0.5968 - val_loss: 6.2639e-05 - val_accuracy: 0.4972
Epoch 4/300
1904/1904 [==============================] - 446s 234ms/step - loss: 1.0112e-05 - accuracy: 0.6035 - val_loss: 6.6294e-05 - val_accuracy: 0.7631
Epoch 5/300
1904/1904 [==============================] - 201s 105ms/step - loss: 1.0277e-05 - accuracy: 0.6173 - val_loss: 6.3263e-05 - val_accuracy: 0.3916
Epoch 6/300
1904/1904 [==============================] - 219s 115ms/step - loss: 9.0461e-06 - accuracy: 0.6459 - val_loss: 6.3458e-05 - val_accuracy: 0.5076
Epoch 7/300
 299/1904 [===>..........................] - E